<p style="font-weight:bold;"> <span style="font-size: 36px"> Contractual Service Margin and Loss Component Switch</span> </p>

<p style="font-weight:bold;"> <span style="font-size: 21px"> Impact of the switch logic to Financial Performance and Balance Sheet</span> </p>

# Theory 
A fundamental concept introduced by IFRS 17 is the Contractual Service Margin (CSM). The CSM represents the unearned profit that an entity expects to earn as it provides services. It is defined at the minimum granularity being the Group of Insurance Contract (Unit Of Account) and it is relevant for all Liability of Remaining Coverage (LRC) approaches with the exception of the Premium Allocation Approach.

The main principles to understand the Contractual Sevice Margin and Loss Component in IFRS 17 are: 

**Principle 1**: When an insurer writes profitable business, it must not be allowed to recognise the expected profits immediately and instead *must spread those profits over time*.

**Principle 2**: When an insurer writes loss-making business, it must not be allowed to spread the expected losses for that business over time and instead *must recognise those losses immediately*.
<br/><br/>
<a name="TOC"></a>
<center><img src="../../Images/CsmLc.png" width="45%" style="float:center;">
<br/><br/><br/>

**Principle 3**: The CSM must be adjusted for all changes (AoC Step) that relate to future service e.g. favourable mortality updates must increase the CSM; unfavourable lapse experience must decrease the CSM.

**Principle 4**: When an insurer recognises that written business, that was previously expected to be profitable, is now expected to be loss-making, e.g. because of changes relating to future service, it must not be allowed to spread the expected losses for that business over time and instead must recognise those losses immediately. It will do so by first extinguishing the CSM and then establishing a Loss Component (LC) in respect of the remaining excess. This represents a switch of the CSM to LC. 
<br/><br/>
<a name="TOC"></a>
<center><img src="../../Images/Switch.png" width="45%" style="float:center;">

# Case Study
In this case study we look at the allocation of the Technical Margin into Contractual Service Margin or Loss Component and its impact on the Financial Performance and Balance Sheet. 
<br> In particular, we are going to consider two possible approaches for the **switch logic**, which determines how the Technical Margin turns from CSM to LC and vice versa throughout the Analysis of Change (AoC). 

For this exercise, we consider two hypothetical GICs with identical inputs: the same parameters, modelled cash flows, actuals, etc... We will then analyze the results obtained by applying two different implementations of the **switch logic** and its implications to financial figures.

We have defined the following GICs:
 - **MZ 1.1 Multiple Switches**: in this case the allocation of the Technical Margin to either CSM or LC can happen multiple times between the Beginning of Period (BoP) and End of Period (EoP). This means whenever the cumulative Technical Margin changes its sign due to the corresponding AoC contributions, a switch from CSM to LC (and vice versa) is observed.
  
 - **MZ 1.2 Single Switch**: in this case the allocation of the Technical Margin to either CSM or LC can happen only once between the Beginning of Period (BoP) and End of Period (EoP). This means only if there is an effective switch (i.e. the cumulative Technical Margin at the Beginning of Period (BoP) and at the End of Period (EoP) have different signs), a single switch is observed. Otherwise there is no switch.

We will look at the relevant reports for the first quarter of year 2021 (2021 3).

It is worth to mention that in our starndard implementation, we consider *In Force* and *New business* contributions seperately as most analysts consider the new business part of the CSM an important Key Performance Indicator (KPI).

# Data Import

In [0]:
#!import "CsmSwitchDataImport"

In [0]:
ifrs17 = new Ifrs17(Workspace, Scopes, Report, Export);

In [0]:
var reportingYear = 2021; 
var reportingMonth = 3;

# Fulfilment Cash Flow (FCF)

We start by looking at the Fulfilment Cash Flows (FCF). Data and results for both cases are exactly the same.

In [0]:
var fulfillmentCashflows = ifrs17.FulfillmentCashflows;
fulfillmentCashflows.ReportingNode = "CH";
fulfillmentCashflows.ReportingPeriod = (reportingYear, reportingMonth);
fulfillmentCashflows.ColumnSlices = new string[]{"GroupOfContract","EstimateType"};
fulfillmentCashflows.DataFilter = null;
(await fulfillmentCashflows.ToReportAsync) with {Height = 750}

# Written Actuals

Aslo the Actuals for both cases are exactly the same.

In [0]:
var writtenActual = ifrs17.WrittenActuals;
writtenActual.ReportingNode = "CH";
writtenActual.ReportingPeriod = (reportingYear, reportingMonth);
writtenActual.ColumnSlices = new string[]{"GroupOfContract"};
writtenActual.DataFilter =  null;
(await writtenActual.ToReportAsync) with {Height = 400}

# LRC Technical Margin (TM)

In the Technical Margin report we present a unified view on the figures that are allocated to either Contractual Service Margin or to Loss Component. 

A negative value for Technical Margin correspond to an Asset character and thus allocated to the CSM. Conversely, a positive value of the Technical Margin represents a Liability character, which is then allocated to Loss Component.  

An Analysis of Change (AoC) report allows one to explain how the closing value (EoP) is obtained from the opening value (BoP). Each change to the previous value (i.e. a delta) is captured in its dedicated AoC Step (Model Correction, Experience Variance, ...).
At each AoC step, the Technical Margin is recognized as Asset-like or Liability-like according to the sign of the accumulation (aggregation) of all figures of the Technical Margin starting from the opening (BoP) to a given AoC step in the chain.

Through the AoC steps it is possible that such aggregation of the Technical Margin oscillates from positive to negative and vice versa several times. How the Technical Margin figures are then allocated to CSM or LC is controlled by the implementation of the **switch logic**.

As we can see below, at this point, the Technical Margin is still the same for both cases considered, as no allocation to either CSM or LC is performed (i.e. the switch logic).

We also notice that the opening balance (BoP) has the same sign as the closing balance (EoP), thus one would not expect any switch intuitively.

However, in this calculation approach the Technical Margin is calculated independently for each novelty (In Force and New Business) and the expected closing balance by novelty have opposite signs. While the In Force is always negative, the New Business is always positive.

Once the novelties are combined, we must look at the result of all contributions together. In our example, the In Force contribution is larger in magnitude than the New Business, thus the combined effect should be negative.
Since the AoC Steps for combined novelties are still not large enough to change the sign (in this case the Experience Adjustment), the closing balance remains negative.  

In [0]:
var technicalMargins = ifrs17.TechnicalMargins;
technicalMargins.ReportingNode = "CH";
technicalMargins.ReportingPeriod = (reportingYear, reportingMonth);
technicalMargins.ColumnSlices = new string[]{"GroupOfContract"};
technicalMargins.DataFilter = null;
(await technicalMargins.ToReportAsync) with {Height = 600}

# Contractual Service Margin / Loss Component

Here, we show the result of two different implementations of the **switch logic** for the two GICs we are considering.

- **MZ 1.1 Multiple Switches**: it is calculated with our standard methodology (**multiple switches**) which allows to switch at every AoC Step with no limitation of amount of switches. 
- **MZ 1.2 Single Switch**: it is calculated with a switch logic (**single switch**) that shows only one switch when the number of total switches is odd (last relevant switch), none otherwise. 

Because only the New Business figures drive the Technical Margin to the Liability side, one can immediately realize the main difference between these two switch choices.
MZ 1.2 does not recognize any Loss Component, as in practice the Technical Margin at the Beginning of Period has the same sign as at the End of Period (both negative).

In this case the **single switch** (MZ 1.2) logic does not show any switch as both opening and closing figures are negative. The New Business contributions are then forced on the CSM side of this report with a negative sign to reflect the fact that they are decreasing the value. 

Contrarily, the **multiple switches** (MZ 1.1) approach allows the New Business to be recognized in the Loss Component (as mentioned above, because we consider the novelties independently). This generates a discrepancy between the In Force (Asset-like allocated to the CSM) and New Business (Liability-like allocated to the Loss Component).
This is resolved in the Combined section of the report where both In Force and New Business figures are considered together.
The Combined Liability AoC Step takes care of adding the two contributions, releasing the Liability-like contributions and allocating the equivalent amount to the CSM. 

In [0]:
var allocatedTechnicalMargins = ifrs17.AllocatedTechnicalMargins;
allocatedTechnicalMargins.ReportingNode = "CH";
allocatedTechnicalMargins.ReportingPeriod = (reportingYear, reportingMonth);
allocatedTechnicalMargins.ColumnSlices = new string[]{"GroupOfContract", "EstimateType"};
allocatedTechnicalMargins.DataFilter = null;
(await allocatedTechnicalMargins.ToReportAsync) with {Height = 700}

# LRC Actuarial
In the LRC Balance Sheet view, the two approaches show the same Beginning of Period and End of Period (as expected) and can only be distinguished by looking at the AoC chains in detail, where indeed they behave differently.

By slicing in the data by EstimateType (i.e. the individual contributions to the LRC), we notice that the New Business section of the CSM of MZ 1.1 is completely missing (as it is allocated to the LC).
However, the Combined Liability AoC Step (effectively a balancing item) in the CSM of MZ 1.1 brings the New Business contribution back to the CSM, netting the effect and restoring the overall closing balance.

In MZ 1.2, since we have never allocated any LC, the New Business contributions appear as negative CSM (as seen in the corresponding report above) and should cancel the FCF for equivalent AoC Steps. Moreover, the Combined Liability AoC Step should be zero, as there is nothing to balance.

In [0]:
var actuarialLrc = ifrs17.ActuarialLrc;
actuarialLrc.ReportingNode = "CH";
actuarialLrc.ReportingPeriod = (reportingYear, reportingMonth);
actuarialLrc.ColumnSlices = new string[]{"GroupOfContract", "EstimateType"};
actuarialLrc.DataFilter = null;
(await actuarialLrc.ToReportAsync) with {Height = 750}

# Financial Performance

As expected, the *Total comprehensive Income* is the same in both scenarios. 
<br> One observes differences in the *Insurance Sevice Result* (ISE) and *Insurance Finance income/Expense* (IFIE) sections. These differences cancel out to produce the same value for *Profit and Loss*. The *Other Comprehensive Income* section reports the same figure in both scenarios. 

Drilling down into the data granularity one can realize the source of the above mentioned differences. 
<br>  Let's start from the IFIE contribution (Financial contributions): 
- MZ 1.1 explicitly recognizes a contribution to Loss Component: this is the interest accretion on the Liability side of the Techinical Margin. Contrarily, MZ 1.2 does not recognizes any Loss Component. 
- MZ 1.1 reports a bigger figure for CSM than MZ 1.2: this is because in MZ 1.2 the interest on the Asset and on the Liability are both allocated to CSM and reported here as summed up (they carry opposite sign - check the CSM/LC report above).

Continuing with the ISE contribution, one sees that both *Insurance Revenue* and *Insurance service Expense* carries a difference between the two scenarios. 
- The difference in *Insurance service Expense* is due to having explicitly allocated the New Business of the Techinical Margin to Loss Component. In this item the financial step (such as Interest Accretion (IA)) is excluded. Thus, because overall the Loss amounts to 0, the net effect of summing all the movements except IA result in $-$IA. This term cancels out the LC contribution reported in the Financial section. 
- The difference in *Insurance Revenue* is produced by the CSM contribution to the *Non Financial LRC changes*. The MZ 1.1 GIC includes here the Combined Liability AoC Step which takes into account the contribution of the IA computed on the New Business (Liability-like).

It is worth to mention that our choice of splitting between financial and non-financial contributions for all figures reported in the Financial Performance creates additional items (rows) that cancel out when aggregated. While for some it might be confusing and counter intuitive, for others this adds transparency. 
We acknowledge that without this split for the Loss Component part, the Financial Performance would not report such items, producing the same final figures without explicit showing any Loss Component contribution.

We also point out that many KPIs can be defined based on some of these figures (CSM and/or LC), and although the Financial Performance in the end is the same, some of these KPIs might look significantly different.

In [0]:
var financialPerformance = ifrs17.FinancialPerformance;
financialPerformance.ReportingNode = "CH";
financialPerformance.ReportingPeriod = (reportingYear, reportingMonth);
financialPerformance.ColumnSlices = new string[]{"GroupOfContract"};
financialPerformance.DataFilter = null;
(await financialPerformance.ToReportAsync) with { Height = 900, GroupDefaultExpanded = 3}